<a href="https://colab.research.google.com/github/slazur83/Advanced_Data_Analysis_in_Oracle_SQL/blob/main/Workouts2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import os
import pandas as pd
import json

# Specify the folder path
folder_path = "/content/drive/MyDrive/Dane z aplikacji/Endomondo/Workouts"

# Initialize an empty list to store individual DataFrames
dfs = []

# Get the list of files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

# Check if there are any JSON files in the folder
if files:
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                # Assuming 'data' is a list of dictionaries
                if isinstance(data, list) and len(data) > 0:
                    sport = 'Running' if data[0].get('sport') == 'RUNNING' else data[0].get('sport')
                    if sport == 'Running':
                        source = data[1].get('source')
                        source = source.replace("INPUT_MANUAL", "Endomondo (Manual)").replace("TRACK_MOBILE", "Endomondo (GPS)")

                        start_time = pd.to_datetime(data[3].get('start_time'))
                        end_time = pd.to_datetime(data[4].get('end_time'))
                        speed_avg_kmh = round(float(data[8].get('speed_avg_kmh')), 2) if data[8].get('speed_avg_kmh') is not None else None
                        duration = int(data[5].get('duration_s')) if data[5].get('duration_s') is not None else None
                        distance = round(float(data[6].get('distance_km')), 2) if duration is not None else None

                        dfs.append({
                            'sport': sport,
                            'source': source,
                            'date': start_time.date(),
                            'start_time': start_time.time(),
                            'end_time': end_time.time(),
                            'duration_s': duration,
                            'distance_km': distance,
                            'speed_avg_kmh': speed_avg_kmh
                        })

            except Exception as e:
                print(f"Error processing JSON file {file_name}: {e}")

    # Create a DataFrame from the list of dictionaries
    df1 = pd.DataFrame(dfs)

    # Sort DataFrame by 'date'
    df1.sort_values(by='date', inplace=True)

    # Reset index
    df1.reset_index(drop=True, inplace=True)

    # Display the sorted and filtered DataFrame
    display(df1)

else:
    print("No JSON files found in the specified folder.")


,sport,source,date,start_time,end_time,duration_s,distance_km,speed_avg_kmh
0,Running,Endomondo (Manual),2012-08-15,10:07:00,10:33:00,1560.0,4.21,9.72
1,Running,Endomondo (Manual),2012-08-18,18:07:00,19:04:00,3420.0,8.48,8.93
2,Running,Endomondo (Manual),2012-08-20,18:30:00,19:19:00,2940.0,6.73,8.24
3,Running,Endomondo (Manual),2012-08-22,18:30:00,19:24:00,3240.0,7.77,8.63
4,Running,Endomondo (Manual),2012-08-26,18:21:00,19:13:00,3120.0,7.77,8.97
...,...,...,...,...,...,...,...,...
646,Running,Endomondo (GPS),2020-10-25,05:47:17,06:46:08,3522.0,10.07,NaN
647,Running,Endomondo (GPS),2020-10-29,14:59:15,16:00:49,3673.0,10.03,NaN
648,Running,Endomondo (GPS),2020-11-01,16:52:07,16:52:10,3239.0,9.44,NaN
649,Running,Endomondo (GPS),2020-11-04,19:53:52,19:53:55,3370.0,9.37,NaN


In [ ]:
#nan_rows = df[df.isna().any(axis=1)]

# Display the DataFrame with NaN values
#display(nan_rows)

In [43]:
import pandas as pd
from datetime import datetime
from dateutil import parser

# File path to the CSV file
file_path = "/content/drive/MyDrive/Dane z aplikacji/MapMyRun/user184238914_workout_history.csv"

# Read the CSV file into a DataFrame using a semicolon as a separator, without a header, and without skipping blank lines
df = pd.read_csv(file_path, header=None, skip_blank_lines=False)

# Filter rows where Activity Type is "Run"
df2 = df[df[2] == 'Run'].copy()

# Rename columns and drop unnecessary ones
df2.rename(columns={1: 'date', 2: 'sport', 4: 'distance_km', 5: 'duration_s', 8: 'speed_avg_kmh', 13: 'source', 14:'url'}, inplace=True)
df2['sport'] = df2['sport'].apply(lambda x: 'Running' if x == 'Run' else x)
df2['source'] = 'Map My Run app'
df2.drop(columns=[0, 3, 6, 7, 9, 10, 11, 12], inplace=True)

# Convert the date column using dateutil.parser
df2['date'] = df2['date'].apply(lambda x: parser.parse(x).strftime('%Y-%m-%d'))
df2['distance_km'] = df2['distance_km'].astype(float).round(2)
df2['speed_avg_kmh'] = df2['speed_avg_kmh'].astype(float).round(2)

df2 = df2[['sport', 'source', 'date', 'duration_s', 'distance_km', 'speed_avg_kmh']]
# Display the DataFrame with the updated column names and format
display(df2)

,sport,source,date,duration_s,distance_km,speed_avg_kmh
1,Running,Map My Run app,2020-12-17,600,1.61,9.66
3,Running,Map My Run app,2020-11-08,3163,9.16,10.43
4,Running,Map My Run app,2020-11-04,3370,9.37,10.01
7,Running,Map My Run app,2020-11-01,3239,9.44,10.49
11,Running,Map My Run app,2020-10-29,3673,10.03,9.83
...,...,...,...,...,...,...
4776,Running,Map My Run app,2012-08-26,3120,7.77,8.97
4777,Running,Map My Run app,2012-08-22,3240,7.77,8.63
4778,Running,Map My Run app,2012-08-20,2940,6.73,8.24
4779,Running,Map My Run app,2012-08-18,3420,8.48,8.93


In [5]:
# Sprawdź, czy istnieją duplikaty w ramce danych
are_duplicates = df_filtered.duplicated().any()

# Wyświetl wynik
print("Czy istnieją duplikaty:", are_duplicates)

Czy istnieją duplikaty: False


In [76]:
merged_df = pd.merge(df1, df2, on=['sport', 'date', 'distance_km'], how='outer', suffixes=('_df1', '_df2'))

# Sprawdź duplikaty na podstawie 'sport', 'date', 'distance_km'
duplicates = merged_df.duplicated(subset=['sport', 'date', 'distance_km'])

# Wybierz wiersze, które nie są duplikatami
result_df = merged_df[~duplicates].copy()  # Skopiuj DataFrame, aby uniknąć SettingWithCopyWarning

# Sortowanie po dacie
result_df['date'] = pd.to_datetime(result_df['date'])
result_df.sort_values(by='date', inplace=True)

# Reset indeksu
result_df.reset_index(drop=True, inplace=True)

# Wyświetl wynik
display(result_df)


,sport,source_df1,date,start_time,end_time,duration_s_df1,distance_km,speed_avg_kmh_df1,source_df2,duration_s_df2,speed_avg_kmh_df2
0,Running,Endomondo (Manual),2012-08-15,10:07:00,10:33:00,1560.0,4.21,9.72,NaN,NaN,NaN
1,Running,NaN,2012-08-15,NaN,NaN,NaN,4.21,NaN,Map My Run app,1560,9.72
2,Running,Endomondo (Manual),2012-08-18,18:07:00,19:04:00,3420.0,8.48,8.93,NaN,NaN,NaN
3,Running,NaN,2012-08-18,NaN,NaN,NaN,8.48,NaN,Map My Run app,3420,8.93
4,Running,Endomondo (Manual),2012-08-20,18:30:00,19:19:00,2940.0,6.73,8.24,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1363,Running,NaN,2020-11-04,NaN,NaN,NaN,9.37,NaN,Map My Run app,3370,10.01
1364,Running,Endomondo (GPS),2020-11-04,19:53:52,19:53:55,3370.0,9.37,NaN,NaN,NaN,NaN
1365,Running,Endomondo (Manual),2020-11-08,17:29:00,17:29:03,3163.0,9.16,10.43,NaN,NaN,NaN
1366,Running,NaN,2020-11-08,NaN,NaN,NaN,9.16,NaN,Map My Run app,3163,10.43
